In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
#%pylab inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import datetime
import io
import os
import sys
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'

# start datajoint using local server
import datajoint as dj
from pose_pipeline.utils.jupyter import play, play_grid

import pose_pipeline
pose_pipeline.set_environmental_variables()
from pose_pipeline.pipeline import *

In [ ]:
# set up tracking method

filt = 'video_project="SPASTICITY" or video_project="SLING" or video_project="AMPUTEE_SAMPLES" or video_project="PARKINSONS" or video_project="PROSTHETIC_GAIT"'

keys = ((pose_pipeline.Video & filt) - pose_pipeline.TrackingBboxMethod).fetch('KEY')

print(f'Missing tracking methods: {len(keys)}')
tracking_method = (pose_pipeline.TrackingBboxMethodLookup & 'tracking_method_name="MMTrack"').fetch1('tracking_method')
# switched from DeepSortYOLOv4 to MMTrack as it seems to be more reliable
for key in keys:
    key['tracking_method'] = tracking_method
    pose_pipeline.TrackingBboxMethod.insert1(key)

pose_pipeline.PersonBbox.populate(filt)
keys = ((pose_pipeline.PersonBbox & filt) - pose_pipeline.TopDownMethod).fetch('KEY')

print(f'Missing top down methods: {len(keys)}')
top_down_method = (pose_pipeline.TopDownMethodLookup & 'top_down_method_name="MMPose"').fetch1('top_down_method')
for key in keys:
    key['top_down_method'] = top_down_method
    pose_pipeline.TopDownMethod.insert1(key)

In [ ]:
pose_pipeline.OpenPose.populate(filt)
pose_pipeline.TrackingBbox.populate(filt)
pose_pipeline.TopDownPerson.populate(filt)